In [1]:
import pandas as pd 
import numpy as np
import sklearn
import keras
from keras import models
from keras import layers
from keras.layers import LSTM
from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.


In [2]:
# import Google Drive 套件
from google.colab import drive

# 將自己的雲端硬碟掛載上去
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
# load models
from keras.models import load_model
Intensity_Classifier = load_model('gdrive/My Drive/金融科技Final_project/Intensity_Classifier')
Small_Event_Classifier = load_model('gdrive/My Drive/金融科技Final_project/Small_Event_Classifier')
Big_Event_Classifier = load_model('gdrive/My Drive/金融科技Final_project/Big_Event_Classifier')
Month_abnormal_returns_Classifier = load_model('gdrive/My Drive/金融科技Final_project/Month_abnormal_returns_Classifier')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [0]:
# load encoders
from joblib import dump, load

Intensity_enc = load('gdrive/My Drive/金融科技Final_project/Intensity_enc.joblib')

BE_enc = load('gdrive/My Drive/金融科技Final_project/BE_enc.joblib')

SE_enc = load('gdrive/My Drive/金融科技Final_project/SE_enc.joblib')

In [0]:
def preprocess_text(corpus, MAX_NUM_WORDS = 10000, MAX_SEQUENCE_LENGTH = 1000):
  tokenizer = keras .preprocessing.text.Tokenizer(num_words=MAX_NUM_WORDS)
  tokenizer.fit_on_texts(corpus)
  output = tokenizer.texts_to_sequences(corpus)
  output = keras .preprocessing .sequence .pad_sequences(output, maxlen=MAX_SEQUENCE_LENGTH)
  return output

In [0]:
def predict_intensity(x):
  pred_Intensity = Intensity_Classifier.predict(x)
  pred_Intensity =(pred_Intensity>0.5).astype(int)

  # decode categorical dummy variables to categorical variables
  pred_Intensity = Intensity_enc.inverse_transform(pred_Intensity)
  return pred_Intensity

In [0]:
def predict_Big_Event(x):
  pred_Big_Event = Big_Event_Classifier.predict(x)
  pred_Big_Event =(pred_Big_Event>0.5).astype(int)

  # decode categorical dummy variables to categorical variables
  pred_Big_Event = BE_enc.inverse_transform(pred_Big_Event)
  return pred_Big_Event

In [0]:
def predict_Small_Event(x):
  pred_Small_Event = Small_Event_Classifier.predict(x)
  pred_Small_Event =(pred_Small_Event>0.5).astype(int)

  # decode categorical dummy variables to categorical variables
  pred_Small_Event = SE_enc.inverse_transform(pred_Small_Event)
  return pred_Small_Event

In [0]:
def predict_Month_abnormal_returns(x):
  pred_Month_abnormal_returns = Month_abnormal_returns_Classifier.predict(x)
  pred_Month_abnormal_returns =(pred_Month_abnormal_returns>0.5).astype(int)
  return pred_Month_abnormal_returns

# Model's summary

In [0]:
Big_Event_Classifier.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 128)         1280000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 16)                9280      
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 85        
Total params: 1,289,365
Trainable params: 1,289,365
Non-trainable params: 0
_________________________________________________________________


In [11]:
Small_Event_Classifier.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, None, 128)         1280000   
_________________________________________________________________
lstm_6 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_15 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_16 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_17 (Dense)             (None, 15)                495       
Total params: 1,333,039
Trainable params: 1,333,039
Non-trainable params: 0
_________________________________________________________________


In [10]:
Intensity_Classifier.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 128)         1280000   
_________________________________________________________________
lstm_3 (LSTM)                (None, 16)                9280      
_________________________________________________________________
dense_3 (Dense)              (None, 7)                 119       
Total params: 1,289,399
Trainable params: 1,289,399
Non-trainable params: 0
_________________________________________________________________


In [0]:
Month_abnormal_returns_Classifier.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 128)         1280000   
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_13 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_14 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_15 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 33        
Total params: 1,333,633
Trainable params: 1,333,633
Non-trainable params: 0
____________________________________________

# Try to use our models

In [0]:
# 透過 gdrive/My Drive/... 來存取檔案
data_df = pd.read_excel('gdrive/My Drive/金融科技Final_project/2019+2020年新聞資訊事件_股價報酬_528.xlsx',sheet_name='year（小事件重分）')

In [0]:
data_df = data_df[0:20]

In [0]:
data_df

,個股代號,公司簡稱,事件日,TCRI(年/月),事件強度,收盤價,明日收盤價 / 昨日收盤價,明日收盤價 / 昨日收盤價 - 1,兩日報酬率(%),前5 ~ 245日平均兩日報酬率(%)，年平均正常報酬,前5 ~ 65日平均兩日報酬率(%)，季平均正常報酬,前5 ~ 25日平均兩日報酬率(%)，月平均正常報酬,前5 ~ 245日(年平均)兩日異常報酬,前5 ~ 65日(季平均)兩日異常報酬,前5 ~ 25日(月平均)兩日異常報酬,大事件類別,小事件類別,事件內容,Unnamed: 18,content_tokenized
0,1218,泰山,20190101,6(2018/09),0,19.00,0.972973,-0.027027,-2.702703,0.072009,-0.126391,0.911512,-2.774712,-2.576311,-3.614215,M_經營層,MT06_高管異動,發言人林俐婉內部調動，由江巍峰接任。。,19.0,發言人 林俐婉 內部 調動 由 江巍峰 接任
1,1503,士電,20190101,4(2018/09),0,40.95,0.997540,-0.002460,-0.246002,-0.000782,-0.459071,-0.042079,-0.245220,0.213068,-0.203923,M_經營層,MT06_高管異動,內部稽核主管林志強內部調動，由莊文清接任。。,22.0,內部 稽核 主管 林志強 內部 調動 由 莊文清 接任
2,1504,東元,20190101,4(2018/09),0,17.45,0.994302,-0.005698,-0.569801,-0.373847,-0.699126,0.436007,-0.195954,0.129325,-1.005807,M_經營層,MT06_高管異動,會計主管藍俊雄內部調動，由林鴻名接任。。,20.0,會計 主管 藍俊雄 內部 調動 由 林鴻名 接任
3,1709,和益,20190101,5(2018/09),0,14.80,1.003367,0.003367,0.336700,-0.097105,-0.364538,-0.160870,0.433805,0.701238,0.497570,M_經營層,MT06_高管異動,內部稽核主管游本詮內部調動，由曾筱茜接任。。,22.0,內部 稽核 主管 游本詮 內部 調動 由 曾筱茜 接任
4,1721,三晃,20190101,7(2018/09),0,10.05,1.000000,0.000000,0.000000,-0.275157,-0.026065,0.557661,0.275157,0.026065,-0.557661,M_經營層,MT06_高管異動,財務經理洪廷宜內部調動，由王婷渝接任。。,20.0,財務 經理 洪廷宜 內部 調動 由 王婷渝 接任
5,1817,凱撒衛,20190101,6(2018/09),0,36.95,0.989276,-0.010724,-1.072386,-0.157922,-0.081807,0.365883,-0.914464,-0.990579,-1.438269,M_經營層,MT06_高管異動,研發主管吳政峰內部調動，由朱清立接任。。,20.0,研發 主管 吳政峰 內部 調動 由 朱清立 接任
6,2064,晉椿,20190101,7(2018/09),-1,15.55,1.016340,0.016340,1.633987,-0.069461,-0.219389,-0.009465,1.703448,1.853376,1.643452,M_經營層,MT06_高管異動,總經理高進義離職，由陳譽接任。。發言人高進義離職，由陳譽接任。。,32.0,總經理 高進義 離職 由 陳譽 接任 發言人 高進義 離職 由 陳譽 接任
7,2207,和泰車,20190101,4(2018/09),0,255.50,0.988000,-0.012000,-1.200000,-0.262075,-0.048627,1.179845,-0.937925,-1.151373,-2.379845,M_經營層,MT02_董監異動,改派1董。董事大野勝仁(豐田自動車代表)卸任。董事長沼一生(豐田自動車代表)新任。,41.0,改派 1 董 董事 大野勝仁 豐田 自動車 代表 卸任 董事長 沼一生 豐田 自動車 代表 新任
8,2330,台積電,20190101,1(2018/09),-1,225.50,0.984305,-0.015695,-1.569507,-0.024201,-0.551439,0.027877,-1.545306,-1.018067,-1.597384,M_經營層,MT02_董監異動,辭任1董。獨立董事湯馬斯?延吉布斯卸任。,20.0,辭任 1 董 獨立 董事 湯馬斯延吉布斯 卸任
9,2357,華碩,20190101,2(2018/09),0,201.50,1.017500,0.017500,1.750000,-0.240916,-0.822804,-0.693053,1.990916,2.572804,2.443053,M_經營層,MT06_高管異動,總經理沈振來內部調動，由胡書賓接任。。,19.0,總經理 沈振來 內部 調動 由 胡書賓 接任


In [0]:
x = data_df.content_tokenized

In [0]:
x.shape

(20,)

In [0]:
x

0                                發言人 林俐婉 內部 調動 由 江巍峰 接任
1                           內部 稽核 主管 林志強 內部 調動 由 莊文清 接任
2                              會計 主管 藍俊雄 內部 調動 由 林鴻名 接任
3                           內部 稽核 主管 游本詮 內部 調動 由 曾筱茜 接任
4                              財務 經理 洪廷宜 內部 調動 由 王婷渝 接任
5                              研發 主管 吳政峰 內部 調動 由 朱清立 接任
6                 總經理 高進義 離職 由 陳譽 接任 發言人 高進義 離職 由 陳譽 接任
7      改派 1 董 董事 大野勝仁 豐田 自動車 代表 卸任 董事長 沼一生 豐田 自動車 代表 新任
8                               辭任 1 董 獨立 董事 湯馬斯延吉布斯 卸任
9                                總經理 沈振來 內部 調動 由 胡書賓 接任
10                                總經理 徐祥 內部 調動 由 江勝昌 接任
11    新美齊 澄清 報載 提及 本 公司 停業 及 隱射 屬 大同 集團 上市櫃 公司 之 報導 ...
12    改派 1 董 董事 周威良 Furama Hotel International Manag...
13                             會計 主管 林慧茹 內部 調動 由 楊崑岳 接任
14                             財務 經理 李易致 內部 調動 由 施冬森 接任
15                             會計 主管 施貽昶 內部 調動 由 呂雅茹 接任
16                               總經理 邱智科 內部 調動 由 林威村 接任
17                            研發 主管 宋漢釧 內部 調動 由 

In [0]:
x = preprocess_text(corpus = data_df.content_tokenized, MAX_NUM_WORDS = 10000, MAX_SEQUENCE_LENGTH = 1305)

y = data_df["大事件類別"]

In [0]:
x = predict_Big_Event(x)

In [0]:
x

array([['I_產業前景'],
       ['I_產業前景'],
       ['I_產業前景'],
       ['A_會計/財報分析'],
       ['I_產業前景'],
       ['I_產業前景'],
       ['I_產業前景'],
       ['F_市場交易'],
       ['M_經營層'],
       ['I_產業前景'],
       ['M_經營層'],
       ['I_產業前景'],
       ['I_產業前景'],
       ['I_產業前景'],
       ['I_產業前景'],
       ['I_產業前景'],
       ['M_經營層'],
       ['I_產業前景'],
       ['M_經營層'],
       ['I_產業前景']], dtype=object)

# Try encoders

In [0]:
BE_y = data_df['大事件類別']
print(BE_y)

BE_y = np.asarray(BE_y)

0     M_經營層
1     M_經營層
2     M_經營層
3     M_經營層
4     M_經營層
5     M_經營層
6     M_經營層
7     M_經營層
8     M_經營層
9     M_經營層
10    M_經營層
11    M_經營層
12    M_經營層
13    M_經營層
14    M_經營層
15    M_經營層
16    M_經營層
17    M_經營層
18    M_經營層
19    M_經營層
Name: 大事件類別, dtype: object


In [0]:
BE_y = BE_enc.transform(BE_y.reshape(-1, 1))
BE_y = BE_enc.inverse_transform(BE_y)

print(BE_y)

[['M_經營層']
 ['M_經營層']
 ['M_經營層']
 ['M_經營層']
 ['M_經營層']
 ['M_經營層']
 ['M_經營層']
 ['M_經營層']
 ['M_經營層']
 ['M_經營層']
 ['M_經營層']
 ['M_經營層']
 ['M_經營層']
 ['M_經營層']
 ['M_經營層']
 ['M_經營層']
 ['M_經營層']
 ['M_經營層']
 ['M_經營層']
 ['M_經營層']]


In [0]:
SE_y = data_df['小事件類別']
print(SE_y)

SE_y = np.asarray(SE_y)

0     MT06_高管異動
1     MT06_高管異動
2     MT06_高管異動
3     MT06_高管異動
4     MT06_高管異動
5     MT06_高管異動
6     MT06_高管異動
7     MT02_董監異動
8     MT02_董監異動
9     MT06_高管異動
10    MT06_高管異動
11       經營層_其他
12    MT02_董監異動
13    MT06_高管異動
14    MT06_高管異動
15    MT06_高管異動
16    MT06_高管異動
17    MT06_高管異動
18    MT06_高管異動
19    MT06_高管異動
Name: 小事件類別, dtype: object


In [0]:
SE_y = SE_enc.transform(SE_y.reshape(-1, 1))
SE_y = SE_enc.inverse_transform(SE_y)

print(SE_y)

[['MT06_高管異動']
 ['MT06_高管異動']
 ['MT06_高管異動']
 ['MT06_高管異動']
 ['MT06_高管異動']
 ['MT06_高管異動']
 ['MT06_高管異動']
 ['MT02_董監異動']
 ['MT02_董監異動']
 ['MT06_高管異動']
 ['MT06_高管異動']
 ['經營層_其他']
 ['MT02_董監異動']
 ['MT06_高管異動']
 ['MT06_高管異動']
 ['MT06_高管異動']
 ['MT06_高管異動']
 ['MT06_高管異動']
 ['MT06_高管異動']
 ['MT06_高管異動']]


In [0]:
Intensity_y = data_df['事件強度']
print(Intensity_y)

Intensity_y = np.asarray(Intensity_y)

0     0
1     0
2     0
3     0
4     0
5     0
6    -1
7     0
8    -1
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
Name: 事件強度, dtype: int64


In [0]:
Intensity_y = Intensity_enc.transform(Intensity_y.reshape(-1, 1))
Intensity_y = Intensity_enc.inverse_transform(Intensity_y)

print(Intensity_y)

[[ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [-1]
 [ 0]
 [-1]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]]
